In [65]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from copy import deepcopy

In [66]:
'''
helper function to help clean-up/format the batch/historical data columns
'''
def custom_date_parser(x):
    try:
        return pd.to_datetime(x, format='%m.%d.%y')
    except ValueError:
        return pd.NaT  # returns 'Not-a-Time' for non-date values

In [67]:
START_DATE_W1 = "Start Date (W1)"
END_DATE_W1 = "End Date (W1)"
START_DATE_W2 = "Start Date (W2)"
END_DATE_W2 = "End Date (W2)"


scheduler_df = pd.read_csv("./FY 24 PD.csv",  parse_dates=[START_DATE_W1, END_DATE_W1, START_DATE_W2, END_DATE_W2], 
                            date_parser=custom_date_parser)
pd.DataFrame(scheduler_df)
scheduler_df

,1=Headline,Program,Ets. # or Final # of Participants,Contract,PM,Start Date (W1),End Date (W1),Start Date (W2),End Date (W2),Faculty,...,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74
0,1.0,Info Session 1 (OE),0,NaN,Yassmine,2023-07-18,2023-07-18,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,Leader as Coach (Offical start in June 23),19,MOU Done!,Kelcie,2023-07-11,2023-07-11,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,Leader as Coach,19,MOU Done!,Kelcie,2023-07-11,2023-07-11,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,Leader as Coach,19,MOU Done!,Kelcie,2023-07-18,2023-07-18,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,Weatherford,30,Done!,Yassmine,2023-07-20,2023-07-21,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,NaN,CoachRICE Online (Spring) M4,TBD,Drafting,Kelcie,2024-06-26,NaT,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
320,NaN,CoachRICE Online (Spring) M4,TBD,Drafting,Kelcie,2024-06-27,NaT,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
321,NaN,CoachRICE Online (Spring) M4,TBD,Drafting,Kelcie,2024-06-28,NaT,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
322,1.0,Anahuac-Inter Coaching,30,No,Megan,NaT,NaT,NaT,NaT,TBD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
# def calculate_available_pms_for_week(week_start, scheduler_df):
#     week_end = week_start + timedelta(days=4)  # Assuming a week goes from Monday to Friday
#     busy_pms = set()

#     for _, program in scheduler_df.iterrows():
#         if not (program['End Date'] < week_start or program['Start Date'] > week_end):
#             busy_pms.add(program['PM'])

#     total_pms = scheduler_df['PM'].nunique()
#     available_pms = total_pms - len(busy_pms)

#     #print(week_start, available_pms)
#     return available_pms

def calculate_available_pms_for_week(week_start, scheduler_df):
    week_end = week_start + timedelta(days=4)  # Assuming a week goes from Monday to Friday
    busy_pms = set()

    # Iterate through each program in the DataFrame
    for _, program in scheduler_df.iterrows():
        # Check if the program overlaps with the week in question for either week 1 or week 2
        overlap_w1 = not (program['End Date (W1)'] < week_start or program['Start Date (W1)'] > week_end)
        overlap_w2 = not (program['End Date (W2)'] < week_start or program['Start Date (W2)'] > week_end)
        
        # If the program overlaps in either week, add the PM to the busy set
        if overlap_w1 or overlap_w2:
            busy_pms.add(program['PM'])

    # Calculate the number of unique PMs and the number of available PMs
    total_pms = scheduler_df['PM'].nunique()
    available_pms = total_pms - len(busy_pms)

    return available_pms


In [69]:
# Assuming scheduler_df is the dataframe containing the program information
# with 'Start Date' and 'End Date' columns as datetime objects

# Assuming 'Start Date' and 'End Date' are in 'MM/DD/YYYY' format
scheduler_df['Start Date (W1)'] = pd.to_datetime(scheduler_df['Start Date (W1)'], format='%m.%d.%y')
scheduler_df['Start Date (W2)'] = pd.to_datetime(scheduler_df['Start Date (W2)'], format='%m.%d.%y')
scheduler_df['End Date (W1)'].fillna(scheduler_df['Start Date (W1)'], inplace=True)
scheduler_df['End Date (W2)'] = pd.to_datetime(scheduler_df['End Date (W2)'], format='%m.%d.%y')

# Find the earliest start date and the latest end date
earliest_start = scheduler_df['Start Date (W1)'].min()
latest_end = scheduler_df[['End Date (W2)', 'End Date (W1)']].max(axis=1).max()

# Find the Monday before the earliest start date
days_until_monday = (earliest_start.weekday() + 1) % 7
first_monday = earliest_start - timedelta(days=days_until_monday)

# Find the Friday after the latest end date
days_until_friday = (4 - latest_end.weekday()) % 7
last_friday = latest_end + timedelta(days=days_until_friday)


# Calculate the total number of weeks
total_weeks = (last_friday - first_monday).days // 7 + 1


# Get the total unique PMs
total_unique_pms = scheduler_df['PM'].nunique()

# Initialize the matrix
hours_matrix = np.full((total_weeks, 5), 7 * total_unique_pms)

for week_index in range(total_weeks):
    week_start = first_monday + timedelta(weeks=week_index)
    available_pms = calculate_available_pms_for_week(week_start, scheduler_df)
    hours_matrix[week_index, :] = 7 * available_pms * 2  # 7 hours per day, multiplied by 2 as per requirement

# Define the function to update the matrix for each program
def update_hours_matrix(matrix, start_date, end_date, first_monday):
    # Calculate program duration in days and initial hours to subtract based on criteria
    program_duration = (end_date - start_date).days + 1
    hours_to_subtract = program_duration * 2 * 7  # Adjust based on specific rules
    week_index = (start_date - first_monday).days // 7
    day_index = min(start_date.weekday(), 4)  # Get the weekday index for the start date

    day_count = 0

    while hours_to_subtract > 0 and week_index >= 0:
    
        # Calculate the subtraction amount for this iteration
        if day_count < 6:
            subtraction_amount = 5  # Starting week special rule
        elif 6 <= day_count <= 10:
            subtraction_amount = 2  # Second week before the program starts
        else:
            subtraction_amount = 1  # From the third week before and onwards
        #print("week,day index", week_index, day_index, hours_to_subtract)
        # Ensure we don't subtract more than what's available or needed
        actual_subtraction = min(subtraction_amount, hours_to_subtract)
        if week_index >= 0 and day_index >= 0:  # Check bounds
            matrix[week_index, day_index] -= actual_subtraction
            hours_to_subtract -= actual_subtraction

        # Move to the previous day, wrapping to the previous week if necessary
        if day_index == 0:
            week_index -= 1
            day_index = 4  # Wrap to Friday of the previous week
        else:
            day_index -= 1  # Move to the previous day within the same week
        day_count += 1
    
# Update the matrix for each program
for index, program in scheduler_df.iterrows():
    update_hours_matrix(hours_matrix, 
                        program['Start Date (W1)'], 
                        program['End Date (W1)'], 
                        first_monday)
    
    update_hours_matrix(hours_matrix, 
                    program['Start Date (W2)'], 
                    program['End Date (W2)'], 
                    first_monday)

# Convert the matrix to a DataFrame for better readability
hours_df = pd.DataFrame(hours_matrix, 
                        index=pd.date_range(first_monday, periods=total_weeks, freq='W-MON'),
                        columns=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'])

print(hours_df)

hours_df.to_csv('hours distribution.csv', index=True)

            Monday  Tuesday  Wednesday  Thursday  Friday
2023-07-10     -16      -16         -6        -9     -19
2023-07-17     -21      -25        -20       -21     -15
2023-07-24     -13      -13        -12       -18     -23
2023-07-31     -35      -36        -35       -20     -23
2023-08-07     -38      -44        -40       -30     -30
...            ...      ...        ...       ...     ...
2028-09-04       0        0          0         0       0
2028-09-11       0        0          0         0       0
2028-09-18       0        0          0         0       0
2028-09-25       0        0          0         0       0
2028-10-02       0        0         -4        -5      -5

[274 rows x 5 columns]


In [70]:
def update_hours_matrix2(matrix, start_date, end_date, first_monday):
    new_matrix = deepcopy(matrix)
    # Calculate program duration in days and initial hours to subtract based on your criteria
    program_duration = (end_date - start_date).days + 1
    hours_to_subtract = program_duration * 2 * 7  # Adjust based on your specific rules
    week_index = (start_date - first_monday).days // 7
    day_index = min(start_date.weekday(), 4)  # Get the weekday index for the start date
  
   # print("start_date", start_date, hours_to_subtract, week_index)
    day_count = 0

    while hours_to_subtract > 0 and week_index >= 0:
    
        # Calculate the subtraction amount for this iteration
        if day_count < 6:
            subtraction_amount = 5  # Starting week special rule
        elif 6 <= day_count <= 10:
            subtraction_amount = 2  # Second week before the program starts
        else:
            subtraction_amount = 1  # From the third week before and onwards
    
        # Ensure we don't subtract more than what's available or needed
        actual_subtraction = min(subtraction_amount, hours_to_subtract)
        if week_index >= 0 and day_index >= 0:  # Check bounds
            new_matrix[week_index, day_index] -= actual_subtraction
            hours_to_subtract -= actual_subtraction

        # Move to the previous day, wrapping to the previous week if necessary
        if day_index == 0:
            week_index -= 1
            day_index = 4  # Wrap to Friday of the previous week
        else:
            day_index -= 1  # Move to the previous day within the same week
        
        day_count += 1
    return new_matrix

In [71]:
PROPOSED_DATE = "11/6/2023"
PROGRAM_DURATION_DAYS = 3



In [72]:
program_start_date = pd.to_datetime(PROPOSED_DATE)

original_hours_matrix = deepcopy(hours_matrix)
print(original_hours_matrix, "\n")

# -- 

matrix = update_hours_matrix2(original_hours_matrix, program_start_date - timedelta(weeks=1), program_start_date - timedelta(weeks=1)+ timedelta(days=PROGRAM_DURATION_DAYS - 1), first_monday)
print(matrix)
before_week_index = ((program_start_date - timedelta(weeks=1)) - first_monday).days // 7
print("before week index", before_week_index)
print("sum row", sum(matrix[before_week_index]))
print(matrix[113])
print(" ")


# ----

matrix = update_hours_matrix2(original_hours_matrix, program_start_date, program_start_date + timedelta(days=PROGRAM_DURATION_DAYS - 1), first_monday)
print(matrix)
during_week_index = (program_start_date - first_monday).days // 7
print("during week index", during_week_index)
print("sum row", sum(matrix[during_week_index]))
print(matrix[114])
print(" ")

#--- 

matrix = update_hours_matrix2(original_hours_matrix, program_start_date + timedelta(weeks=1), program_start_date + timedelta(weeks=1, days=PROGRAM_DURATION_DAYS - 1), first_monday)
print(matrix)
after_week_index = ((program_start_date + timedelta(weeks=1)) - first_monday).days // 7
print("after week index", after_week_index)
print("sum row", sum(matrix[after_week_index]))
print(matrix[115])
print(" ")

# ---




[[-16 -16  -6  -9 -19]
 [-21 -25 -20 -21 -15]
 [-13 -13 -12 -18 -23]
 ...
 [  0   0   0   0   0]
 [  0   0   0   0   0]
 [  0   0  -4  -5  -5]] 

[[-16 -16  -6  -9 -19]
 [-21 -25 -20 -21 -15]
 [-13 -13 -12 -18 -23]
 ...
 [  0   0   0   0   0]
 [  0   0   0   0   0]
 [  0   0  -4  -5  -5]]
before week index 16
sum row -134
[0 0 0 0 0]
 
[[-16 -16  -6  -9 -19]
 [-21 -25 -20 -21 -15]
 [-13 -13 -12 -18 -23]
 ...
 [  0   0   0   0   0]
 [  0   0   0   0   0]
 [  0   0  -4  -5  -5]]
during week index 17
sum row -235
[0 0 0 0 0]
 
[[-16 -16  -6  -9 -19]
 [-21 -25 -20 -21 -15]
 [-13 -13 -12 -18 -23]
 ...
 [  0   0   0   0   0]
 [  0   0   0   0   0]
 [  0   0  -4  -5  -5]]
after week index 18
sum row -55
[0 0 0 0 0]
 
